### Import necessary libraries, packages, and dependencies 
- Use os to access the operating system
- Use pandas to create DataFrames and perform analysis 
- Use plotly to generate visuals
- Use dotenv to access .env variables 
- Use sqlalchemy to connect to postgres

In [1]:
import os
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Connect to database and retrieve necessary data
- Connect to the 'arbi_db' database 
- Retrieve all data from the 'control' table

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access the variables 
psql_username = os.getenv('PSQL_USERNAME')
psql_password = os.getenv('PSQL_PASSWORD')
psql_host = os.getenv('PSQL_HOST')
psql_port = os.getenv('PSQL_PORT')
db_name = os.getenv('DB_NAME')

# Define the database url
db_url = f"postgresql://{psql_username}:{psql_password}@{psql_host}:{psql_port}/{db_name}" 

# Create the engine object
engine = create_engine(db_url)

# Write ethe SQL query
query = 'SELECT * FROM control'

# Read the SQL query into a DataFrame
control_df = pd.read_sql(query, engine)

# Show the DataFrame's head
control_df.head()


,trade_count,current_datetime,currency,volume,buy_exchange,buy_price,total_purchase_amount,sell_exchange,sell_price,total_sale_amount,profit,spread_percentage
0,1,2024-02-28T09:44:38.359438,bitcoin,1.0,Binance,60335.08,60335.08,Bitstamp,60400.00,60400.00,64.92,0.107599
1,2,2024-02-28T09:44:38.359438,bitcoin,1.0,Binance,60335.08,60335.08,Gemini,60390.16,60390.16,55.08,0.091290
2,3,2024-02-28T09:44:38.359438,bitcoin,1.0,Gemini,60394.39,60394.39,Bitstamp,60400.00,60400.00,5.61,0.009289
3,4,2024-02-28T09:44:38.359438,bitcoin,1.0,Kraken,60321.50,60321.50,Bitstamp,60400.00,60400.00,78.50,0.130136
4,5,2024-02-28T09:44:38.359438,bitcoin,1.0,Kraken,60321.50,60321.50,Gemini,60390.16,60390.16,68.66,0.113823


### Data Analysis
- Profit Distribution across Trades
- Hourly Profit Trend
- Spread Percentage Analysis 
- Exchange Performance 
- Time Series Analysis

-Note: Limited data collection - Program ran every 30 seconds over a 4 hour window

In [3]:
# Convert the current_datetime column to only show the date
control_df['current_datetime'] = pd.to_datetime(control_df['current_datetime'])
control_df['current_datetime'] = control_df['current_datetime'].apply(lambda x: x.time())

# Set the index to the current_datetime column
control_df.set_index('current_datetime', inplace=True)

control_df

,trade_count,currency,volume,buy_exchange,buy_price,total_purchase_amount,sell_exchange,sell_price,total_sale_amount,profit,spread_percentage
current_datetime,,,,,,,,,,,
09:44:38.359438,1,bitcoin,1.0,Binance,60335.08000,60335.08000,Bitstamp,60400.00000,60400.00000,64.92000,0.107599
09:44:38.359438,2,bitcoin,1.0,Binance,60335.08000,60335.08000,Gemini,60390.16000,60390.16000,55.08000,0.091290
09:44:38.359438,3,bitcoin,1.0,Gemini,60394.39000,60394.39000,Bitstamp,60400.00000,60400.00000,5.61000,0.009289
09:44:38.359438,4,bitcoin,1.0,Kraken,60321.50000,60321.50000,Bitstamp,60400.00000,60400.00000,78.50000,0.130136
09:44:38.359438,5,bitcoin,1.0,Kraken,60321.50000,60321.50000,Gemini,60390.16000,60390.16000,68.66000,0.113823
...,...,...,...,...,...,...,...,...,...,...,...
12:44:36.959571,7609,xrp,1.0,Bitstamp,0.56593,0.56593,Gemini,0.56609,0.56609,0.00016,0.028272
12:44:36.959571,7610,xrp,1.0,Kraken,0.56587,0.56587,Gemini,0.56609,0.56609,0.00022,0.038878
12:44:36.959571,7611,xrp,1.0,Poloniex,0.56450,0.56450,Bitstamp,0.56517,0.56517,0.00067,0.118689


##### Profit Distribution across Trades
- Calculate and visualize the distribution of profits across all trades. This provides insights into the consistency and variability of returns. 

In [4]:
profit_dist = px.histogram(
    control_df, 
    x='profit', 
    nbins=20, 
    title='Profit Distribution', 
    labels={'profit': 'Profit', 'count':'Frequency'})

profit_dist.show()

##### 30 Sec Profit Trend
- Examine the 30 second trend of profits to identify patterns and understand the overall performance of the arbitrage over time.

In [5]:
thirty_sec_profit = control_df.groupby('current_datetime')['profit'].sum().reset_index()
thirty_sec_profit_plot = px.line(
    thirty_sec_profit, 
    x='current_datetime',
    y='profit', 
    title='30 Second Profit Trend',
    labels={'current_datetime':'Time', 'profit':'30 Sec Profit'})

thirty_sec_profit_plot.show()

##### Spread Percentage Analysis
- Explore the spread percentage to understand how much profit is generated relative to the spread. Helps to access the efficieny of the arbitrage strategy. 

In [6]:
spread_percentage_plot = px.scatter(
    control_df, 
    x='spread_percentage',
    y='profit',
    title='Spread Percentage vs Profit',
    labels={'spread_percentage':'Spread Percentage', 'profit':'Profit'})

spread_percentage_plot.show()

##### Exchange Performance
- Evaluate the performance of each exchange in terms of the number of trades and profit. This helps in optimizing the selection of exchanges and diversifying risks.

In [7]:
exchange_performance = control_df.groupby('buy_exchange')['profit'].agg(['count', 'sum']).reset_index()
exchange_performance_plot = px.bar(
    exchange_performance,
    x='buy_exchange',
    y=['count', 'sum'],
    barmode='stack',
    title='Exchange Performance',
    labels={'buy_exchange':'Exchange', 'value':'Value'}
)

exchange_performance_plot.show()

##### Average Spread Percentage by Coin
- Calculate the average spread percentge for each coin to identify coins with more favorable arbitrage opportunities. This helps in optimizing the selection of coins and diversifying risks.

In [8]:
avg_spread_by_coin = control_df.groupby('currency')['spread_percentage'].mean().reset_index()
avg_spread_by_coin_plot = px.bar(
    avg_spread_by_coin,
    x='currency',
    y='spread_percentage',
    title='Average Spread Percentage for Each Coin',
    labels={'spread_percentage':'Average Spread Percentage', 'currency':'Coin'}
)

avg_spread_by_coin_plot.show()

##### Time Series Analysis
- Analyze the time series data to identify any patterns or trends in the arbitrage opportunities.

In [9]:
time_series = control_df.reset_index()

time_series_plot = px.line(
    time_series,
    x='current_datetime', 
    y='profit', 
    title='Time Series Analysis',
    labels={'current_datetime': 'Time', 'profit':'Profit'}
)
time_series_plot.show()

### Evaluate minimum profit thresholds
- Add transaction fees (use 0.1 percent)
- Adjust the thresholds to account for additonal fees, slippage, etc.

In [10]:
# Calculate total transaction cost for each trade
control_df['buy_fee'] = 0.1  
control_df['sell_fee'] = 0.1  
control_df['total_transaction_cost'] = control_df['buy_fee'] + control_df['sell_fee']

# Filter trades based on a minimum profit threshold in percentage
# Create a list of thresholds values at (.1%, .25%, .5%, .75%, 1%, 3%, 5%)
thresholds = [0.1, 0.25, 0.5, 0.75, 1, 3, 5]

# Create an empty dictionary to store total profits for each threshold
threshold_profits = {}

# Loop through each threshold and calculate profits
for threshold in thresholds:
    # Adjust spread percentage for expected fees
    control_df['adjusted_spread_percentage'] = control_df['spread_percentage'] - control_df['total_transaction_cost']
    
    # Filter trades by threshold
    profitable_trades = control_df[control_df['adjusted_spread_percentage'] > threshold]

    # Calculate profits 
    profitable_trades['profit'] = profitable_trades['sell_price'] - profitable_trades['buy_price'] - profitable_trades['total_transaction_cost']
    total_profits = profitable_trades['profit'].sum()

    # Store results to dictionary 
    threshold_profits[f'{threshold}% Threshold'] = total_profits

/var/folders/wl/x3hxw0v13sb9sgxcqzzdxwlh0000gn/T/ipykernel_62897/40241085.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
# View profits at various thresholds
threshold_profits

{'0.1% Threshold': 329414.23875000014,
 '0.25% Threshold': 290820.72071000014,
 '0.5% Threshold': 144099.82715999996,
 '0.75% Threshold': 70701.46802999996,
 '1% Threshold': 35706.93693999995,
 '3% Threshold': -1.7803400000000005,
 '5% Threshold': 0.0}